<a href="https://colab.research.google.com/github/CarolineWeeLab/EZgut/blob/main/MaskPrediction_FluorescenceAnalysis_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Segmentations of the gut are generated by our custom-trained Pointrend model. This notebook does not include the curation of predicted masks, but if critical errors are discovered upon visualisation of the masks, we suggest that they are editted in ImageJ using selection and ROI tools. The corrected masks can then be replaced in the folder to proceed with fluorescence analysis.

We make use of free Google Colab's GPU runtime so that the notebook can be run on a basic laptop with internet connection, without the need for a local GPU.


---



Updated: 16 March 2022

# Install packges

We use Detectron2's [Pointrend](https://github.com/facebookresearch/detectron2/tree/main/projects/PointRend) model and train it further on out custom dataset.

In [ ]:
# make sure to connect to GPU run time

!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Load data

In [ ]:
# mount drive and load data
# data should be google drive

from google.colab import drive
drive.mount('/content/drive')

# Mask generation

Please ensure that the model file has been downloaded and that the correct path is used.

In [ ]:
# Obtain predictions and save masks

cfg = get_cfg()

################################################################################
#EDIT THIS PART ONLY

cfg.MODEL.WEIGHTS = "model_final.pth"  # path to the model 

################################################################################

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("gut_train",)
cfg.DATASETS.TEST = ("gut_test")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1200    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # minimum detection score for object to be considered
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5 # NMS for boxes with IOU > thresh
predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode

gut_metadata = MetadataCatalog.get("gut_train_v3")


To generate and save masks of the brightfield images. 
directory is where BF images are stored. 
savedir is where output masks will be saved. Both tif and jpg formats are allowed in this model. To use jpg, edit code to filename.endswith("jpg")

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

################################################################################
#EDIT THIS PART ONLY

directory = "" # directory containing the brightfield images
savedir = "" # directory to save predicted masks

################################################################################

for filename in os.listdir(directory):
    if filename.endswith("tif"): # if performance is poor, consider enhancing brightness + contrast in ImageJ
        im = cv2.imread(os.path.join(directory,filename))
        outputs = predictor(im)
        mask = outputs["instances"].pred_masks.to("cpu").numpy() 
        msks = []
        # each instance is a separate mask so mask size is n_instances*image_height*image_width
        for m in mask:
            msks.append(m)
            ms = np.sum(msks[:], axis=0) # compiles all the instances in one image to 1 binary mask image
            ms = np.asarray(ms, np.float)
            m,M = ms.min(), ms.max()
            I = np.asarray((ms - m) / (M - m + 0.000001) * 65535, np.uint16) 
            I = np.where(I < 1,0,65535)
            I = np.asarray(I, dtype=np.int8)
            I= Image.fromarray(I).convert('L') 
            I.save(os.path.join(savedir,filename + '_mask.png')) 

# Visualisation

To check prediction masks of one image or for visualisation

In [ ]:
from PIL import Image

im = cv2.imread("BF_testimage.tif")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=gut_metadata, 
                scale=0.8, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2_imshow(v.get_image()[:, :, ::-1])

To check prediction masks of all images in folder, not recommended if there a lot of images

In [ ]:
################################################################################
#EDIT THIS PART ONLY

directory = ""

################################################################################

for filename in os.listdir(directory):
    if filename.endswith("tif"):
        im = cv2.imread(os.path.join(directory,filename))
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                metadata=gut_metadata, 
                scale=0.8, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        cv2_imshow(v.get_image()[:, :, ::-1])

# Fluorescence analysis

Fill in ch1name as the channel name of your fluorescence channel. We use Cy5.

Your file hierachy should look like (results.csv is created by this cell):

```
maindir
│   
└───BFmask
│   │   BF_sample1_mask.png
|   |   BF_sample2_mask.png
│   
└───BF
|   │   BF_sample1.tif
|   |   BF_sample2.tif
|   
└───Cy5
|   │   Cy5_sample1.tif
|   |   Cy5_sample2.tif
|
|   results.csv
```



In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

################################################################################
#EDIT THIS PART ONLY

ch1name = "Cy5"
maindir = "" # directory where all the BFmask, Cy5 etc. are stored

################################################################################

BFmaskdir = os.path.join(maindir,"BFmask")
ch1 = os.path.join(maindir,ch1name)

results_dict = {}

for im in os.listdir(BFmaskdir):
    if im.endswith('png'):

        maskim = cv.imread(os.path.join(BFmaskdir,im),0)
        if len(maskim.shape) != 2:
            raise Exception("mask image has more than 1 channel")

        imname = im.split('--')[0]
        ch1_im_name = [filename for filename in os.listdir(ch1) if filename.startswith(imname)][0]
        intensityim_ch1 = cv.imread(os.path.join(ch1,ch1_im_name),-1)
        if intensityim_ch1.dtype != 'uint16':
            #raise Exception('Channel 1 image is not 16 bit')
            print('Channel 1 image is not 16 bit')

        labeledim = cv.connectedComponentsWithStats(maskim)
        fishnum = labeledim[0] - 1 # subtract 1 for background

        ch1_mean_list = []
        for i in range(1,fishnum + 1):
            gutmask = labeledim[1] == i
            gutintensity_ch1 = gutmask * intensityim_ch1
            ch1mean_temp = gutintensity_ch1[gutintensity_ch1!=0].mean()
            ch1_mean_list.append(ch1mean_temp)
            results_dict[imname] = ch1_mean_list

results_df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in results_dict.items()]))
results_df.to_csv(os.path.join(maindir,'results.csv'),index=False)

